# Extrair XML

Script que extrai informações de um arquivo .xml de notas fiscais.

In [1]:
# Import das bibliotecas necessárias

import xml.etree.ElementTree as ET  # Para extrair dados das tags do XML
import pandas as pd  # Para manipular dataframes
import os  # Para manipular os arquivos do SO
import warnings  # Para ocultar os warnings
warnings.filterwarnings('ignore')  # Para ocultar os warnings
from openpyxl import load_workbook  # Para abrir arquivos Excel
from openpyxl.worksheet.table import Table, TableStyleInfo  # Para formatar a planilha excel em tabela e editar seu estilo
from openpyxl.utils import get_column_letter  # Para manipular as colunas da tabela Excel
from openpyxl.styles import Font  # Para alterar as fontes do planilha

In [5]:
# Extrair produtos e preços de notas fiscais para arquivos Excel
# Autor: Marcos Paulo Ferreira Gonçalves
# Data: 2023-10-03
# Descrição: Este script percorre arquivos XML de notas fiscais, extrai informações de produtos e preços, e gera um arquivo Excel formatado.

# Lista para armazenar os dados extraídos
lista = []

# Caminho da pasta dos XMLs
pasta_xml = 'XML'

# Caminho do arquivo de referência
caminho_codigo_produto = os.path.join(pasta_xml, 'codigo_produto.xlsx')

# Lê a tabela de referência de códigos
df_codigos = pd.read_excel(caminho_codigo_produto, dtype={'ID_PROD': str, 'ID_SECAO': str})

# Loop pelos XMLs
for arq in os.listdir(pasta_xml):
    if arq.endswith('.xml'):
        tree = ET.parse(os.path.join(pasta_xml, arq))
        root = tree.getroot()
        
        ns = {'nfe': 'http://www.portalfiscal.inf.br/nfe'}
        
        for det in root.findall('.//nfe:det', ns):
            quantidade = det.find('.//nfe:qCom', ns).text
            Cod = det.find('.//nfe:cProd', ns).text
            Prod = det.find('.//nfe:xProd', ns).text
            Vlr = det.find('.//nfe:vUnCom', ns).text

            # Limpeza do código: remove tudo a partir do ponto
            if '.' in Cod:
                Cod = Cod.split('.')[0]
            
            lista.append((Cod, Prod, float(Vlr)))

# Cria DataFrame com os produtos extraídos
df = pd.DataFrame(lista, columns=['Código', 'Produto', 'Valor Unitário'])

# Faz o merge com a tabela de referência para adicionar o ID_SECAO
df_merged = df.merge(df_codigos, how='left', left_on='Código', right_on='ID_PROD')

# Renomeia a coluna "ID_SECAO" para "Seção"
df_merged = df_merged.rename(columns={'ID_SECAO': 'Seção'})

# Seleciona e reorganiza as colunas desejadas
df_merged = df_merged[['Código', 'Seção', 'Produto', 'Valor Unitário']]

# Ordena o DataFrame por Seção, Produto e Valor Unitário
df_merged = df_merged.sort_values(by=['Seção', 'Produto', 'Valor Unitário'], ascending=[True, True, False])

# Salva o Excel (sem formatação ainda)
df_merged.to_excel(pasta_xml+'/produtos.xlsx', index=False)

# Abre o Excel para formatação
wb = load_workbook(pasta_xml+'/produtos.xlsx')
ws = wb.active

# Formato contábil manual brasileiro
formato_contabil_br = 'R$ #,##0.00_);[Red]R$ (#,##0.00)'

# Aplica formatação na coluna "Valor Unitário" (coluna D = 4)
for row in ws.iter_rows(min_row=2, min_col=4, max_col=4):
    for cell in row:
        cell.number_format = formato_contabil_br

# Aplica cor branca na fonte dos títulos (linha 1)
for cell in ws[1]:
    cell.font = Font(color="FFFFFF", bold=True)

# Define a área da tabela
max_linha = ws.max_row
max_coluna = ws.max_column
coluna_final = get_column_letter(max_coluna)
ref_tabela = f"A1:{coluna_final}{max_linha}"

# Cria e estiliza a tabela do Excel
tabela = Table(displayName="TabelaProdutos", ref=ref_tabela)

estilo = TableStyleInfo(
    name="TableStyleMedium16",  # Estilo moderno com zebra
    showFirstColumn=False,
    showLastColumn=False,
    showRowStripes=True,
    showColumnStripes=False
)
tabela.tableStyleInfo = estilo
ws.add_table(tabela)

# Ajusta a largura automática das colunas com base no conteúdo
for coluna in ws.columns:
    max_largura = 0
    for cell in coluna:
        if cell.value:
            max_largura = max(max_largura, len(str(cell.value)))
    col_letter = get_column_letter(cell.column)
    ws.column_dimensions[col_letter].width = max_largura + 2  # margem extra

# Salva o Excel com tudo aplicado
wb.save(pasta_xml+'/produtos.xlsx')